In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string

In [2]:
df = pd.read_csv(r"listings.csv")

C:\Users\jpran\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
#compute the count and mean value as group by the products
count = df.groupby("id", as_index=False).count()
mean = df.groupby("id", as_index=False).mean()

#merge two dataset create df1
df1 = pd.merge(df, count, how='right', on=['id'])

df1.head()

,id,listing_url_x,scrape_id_x,last_scraped_x,name_x,summary_x,space_x,description_x,experiences_offered_x,neighborhood_overview_x,...,instant_bookable_y,is_business_travel_ready_y,cancellation_policy_y,require_guest_profile_picture_y,require_guest_phone_verification_y,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes_y,calculated_host_listings_count_private_rooms_y,calculated_host_listings_count_shared_rooms_y,reviews_per_month_y
0,13913,https://www.airbnb.com/rooms/13913,20190710134553,2019-07-11,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,1,1,1,1,1,1,1,1,1,1
1,15400,https://www.airbnb.com/rooms/15400,20190710134553,2019-07-11,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,...,1,1,1,1,1,1,1,1,1,1
2,17402,https://www.airbnb.com/rooms/17402,20190710134553,2019-07-11,Superb 3-Bed/2 Bath & Wifi: Trendy W1,"Open from June 2018 after a 3-year break, we a...",Ready again from June 2018 for bookings after ...,"Open from June 2018 after a 3-year break, we a...",none,"Location, location, location! You won't find b...",...,1,1,1,1,1,1,1,1,1,1
3,24328,https://www.airbnb.com/rooms/24328,20190710134553,2019-07-11,Battersea bedroom & office w parking shared ga...,"Artist house, bright high ceiling rooms, priva...",- End of terrace two bedroom house close to So...,"Artist house, bright high ceiling rooms, priva...",family,"- Battersea is a quiet family area, easy acces...",...,1,1,1,1,1,1,1,1,1,1
4,25023,https://www.airbnb.com/rooms/25023,20190710134553,2019-07-11,All-comforts 2-bed flat near Wimbledon tennis,"Large, all comforts, 2-bed flat; first floor; ...",10 mins walk to Southfields tube and Wimbledon...,"Large, all comforts, 2-bed flat; first floor; ...",none,This is a leafy residential area with excellen...,...,1,1,1,1,1,1,1,1,1,1


In [48]:
len(df.id.unique())

83850

In [49]:
#rename column
# df1["Count"] = df1["id_x"]
df1["score"] = df1["review_scores_rating_x"]
df1["Summary"] = df1["summary_x"]

#Create New datafram with selected variables
df1 = df1[['id','Summary','score']]

In [50]:
df1.head()

,id,Summary,score
0,13913,My bright double bedroom with a large window h...,96.0
1,15400,Lots of windows and light. St Luke's Gardens ...,96.0
2,17402,"Open from June 2018 after a 3-year break, we a...",94.0
3,24328,"Artist house, bright high ceiling rooms, priva...",98.0
4,25023,"Large, all comforts, 2-bed flat; first floor; ...",91.0


In [7]:
# #create new dataframe as combining all summary with same  Id
# df4 = df1.groupby("id", as_index=False).mean()
# combine_summary = df1.groupby("id")["Summary"].apply(list)
# combine_summary = pd.DataFrame(combine_summary)
# # combine_summary.to_csv("combine_summary.csv")

In [51]:
df1['Summary'].astype(str)

0        My bright double bedroom with a large window h...
1        Lots of windows and light.  St Luke's Gardens ...
2        Open from June 2018 after a 3-year break, we a...
3        Artist house, bright high ceiling rooms, priva...
4        Large, all comforts, 2-bed flat; first floor; ...
5        Big room with double bed/ clean sheets/ clean ...
6        Contemporary design meets classic London in th...
7        Contemporary design meets classic London in th...
8        The name says it all, it really is a restful r...
9        Walking distance to Twickenham Stadium, 35 min...
10       Bright, large dbl room with a very high ceilin...
11       3 Bed House with garden close to Thames river ...
12                                                     nan
13       Situated literally in London Fields park and o...
14                                                     nan
15       The bright, light and stylish apartment in Can...
16       Our guest room & bathroom overlooks the garden.

In [52]:
df1= df1.sample(20000)

In [55]:
df1.dropna(inplace=True)

In [56]:
len(df1)

14046

In [57]:
df1.Summary.isnull()

12394    False
29771    False
36956    False
28378    False
75971    False
21150    False
3473     False
42872    False
16424    False
21208    False
69247    False
31877    False
47519    False
46010    False
36808    False
30653    False
43444    False
39176    False
66036    False
69837    False
78628    False
22554    False
36960    False
57213    False
36825    False
15787    False
9520     False
57671    False
77709    False
15896    False
         ...  
50263    False
22064    False
17707    False
5377     False
7581     False
26862    False
33899    False
73825    False
45589    False
80277    False
50997    False
14018    False
10375    False
31954    False
51908    False
8003     False
43759    False
47555    False
29072    False
75506    False
16855    False
69876    False
71962    False
51090    False
55146    False
36178    False
69546    False
28613    False
57641    False
11534    False
Name: Summary, Length: 14046, dtype: bool

In [58]:
cleanup_re = re.compile('[^a-z]+')
def cleanup(sentence):
    sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

In [59]:
df1["Summary_Clean"] = df1["Summary"].apply(cleanup)

In [60]:
df1.head()

,id,Summary,score,Summary_Clean
12394,8403610,"Clean bright double room to rent in Neasden, 1...",95.0,clean bright double room to rent in neasden mi...
29771,17859972,My place is good for solo adventurers and fami...,100.0,my place is good for solo adventurers and fami...
36956,20465451,My Clapton flat is a spacious and relaxing bas...,91.0,my clapton flat is a spacious and relaxing bas...
28378,17264314,Lovely lower ground floor studio apartment IN ...,87.0,lovely lower ground floor studio apartment in ...
75971,34734783,"A bright, modern, 1 bedroom apartment in the h...",75.0,a bright modern bedroom apartment in the heart...


In [61]:
#df1 = df1.drop_duplicates(['score'], keep='last')
df1 = df1.reset_index()
len(df1)

14046

In [62]:
from pandas import DataFrame 

docs = df1["Summary_Clean"] 
vect = CountVectorizer(max_features = 200, stop_words='english') 
X = vect.fit_transform(docs) 

df5 = DataFrame(X.A, columns=vect.get_feature_names())
df5 = df5.astype(int)
df5

,access,adventurers,airport,airy,amazing,amenities,apartment,area,attractions,available,...,victorian,views,walk,walking,welcome,west,wharf,wifi,wimbledon,zone
0,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# First let's create a dataset called X
X = np.array(df5)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(df5)))
df5_train = X[:tsize]
df5_test = X[tsize:]
#len of train and test
lentrain = len(df5_train)
lentest = len(df5_test)

In [64]:
lentest

1405

In [65]:
# Next we will instantiate a nearest neighbor object, and call it nbrs. Then we will fit it to dataset X.
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(df5_train)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
distances, indices = nbrs.kneighbors(df5_train)

In [66]:
#find most related property - This is for the total number in test data
for i in range(lentest):
    a = nbrs.kneighbors([df5_test[i]])
    related_product_list = a[1]
    
    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Based on listing reviews, for ", df1["id"][lentrain + i] ," and this  Score is ",df1["score"][lentrain + i])
    print ("The first similar property is ", df1["id"][first_related_product] ," and this  Score is ",df1["score"][first_related_product])
    print ("The second similar property is ", df1["id"][second_related_product] ," and this  Score is ",df1["score"][second_related_product])
    print ("-----------------------------------------------------------")

Based on listing reviews, for  4642160  and this  Score is  94.0
The first similar property is  8840980  and this  Score is  99.0
The second similar property is  33388463  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  239697  and this  Score is  94.0
The first similar property is  30707696  and this  Score is  60.0
The second similar property is  33310193  and this  Score is  80.0
-----------------------------------------------------------
Based on listing reviews, for  9224280  and this  Score is  90.0
The first similar property is  18660727  and this  Score is  100.0
The second similar property is  33494490  and this  Score is  90.0
-----------------------------------------------------------
Based on listing reviews, for  34301785  and this  Score is  60.0
The first similar property is  25034577  and this  Score is  85.0
The second similar property is  25609871  and this  Score is  60.0
---------------------------

The first similar property is  8776941  and this  Score is  93.0
The second similar property is  35601216  and this  Score is  20.0
-----------------------------------------------------------
Based on listing reviews, for  23665982  and this  Score is  92.0
The first similar property is  24182179  and this  Score is  98.0
The second similar property is  14907588  and this  Score is  94.0
-----------------------------------------------------------
Based on listing reviews, for  31605663  and this  Score is  97.0
The first similar property is  20467350  and this  Score is  100.0
The second similar property is  21618536  and this  Score is  94.0
-----------------------------------------------------------
Based on listing reviews, for  15669093  and this  Score is  90.0
The first similar property is  32399251  and this  Score is  100.0
The second similar property is  31146548  and this  Score is  73.0
-----------------------------------------------------------
Based on listing reviews, for

Based on listing reviews, for  7436164  and this  Score is  100.0
The first similar property is  318867  and this  Score is  98.0
The second similar property is  34585521  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  23364140  and this  Score is  100.0
The first similar property is  31158604  and this  Score is  80.0
The second similar property is  6669426  and this  Score is  90.0
-----------------------------------------------------------
Based on listing reviews, for  33481290  and this  Score is  97.0
The first similar property is  19883754  and this  Score is  100.0
The second similar property is  36027634  and this  Score is  80.0
-----------------------------------------------------------
Based on listing reviews, for  12344005  and this  Score is  97.0
The first similar property is  13996117  and this  Score is  100.0
The second similar property is  27768781  and this  Score is  80.0
-----------------------

Based on listing reviews, for  21444715  and this  Score is  100.0
The first similar property is  32358738  and this  Score is  80.0
The second similar property is  19805249  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  14276103  and this  Score is  96.0
The first similar property is  8022886  and this  Score is  100.0
The second similar property is  27035574  and this  Score is  80.0
-----------------------------------------------------------
Based on listing reviews, for  34285757  and this  Score is  95.0
The first similar property is  35383322  and this  Score is  100.0
The second similar property is  11210127  and this  Score is  90.0
-----------------------------------------------------------
Based on listing reviews, for  14175546  and this  Score is  97.0
The first similar property is  29645810  and this  Score is  90.0
The second similar property is  7789861  and this  Score is  94.0
----------------------

The first similar property is  15697029  and this  Score is  90.0
The second similar property is  3132302  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  2854777  and this  Score is  100.0
The first similar property is  16272953  and this  Score is  96.0
The second similar property is  7670565  and this  Score is  90.0
-----------------------------------------------------------
Based on listing reviews, for  20020795  and this  Score is  95.0
The first similar property is  32324387  and this  Score is  91.0
The second similar property is  5370860  and this  Score is  77.0
-----------------------------------------------------------
Based on listing reviews, for  7707975  and this  Score is  99.0
The first similar property is  8125962  and this  Score is  97.0
The second similar property is  5442063  and this  Score is  95.0
-----------------------------------------------------------
Based on listing reviews, for  2690

...

Based on listing reviews, for  32692172  and this  Score is  87.0
The first similar property is  16464777  and this  Score is  53.0
The second similar property is  452538  and this  Score is  94.0
-----------------------------------------------------------
Based on listing reviews, for  34663461  and this  Score is  100.0
The first similar property is  26264142  and this  Score is  100.0
The second similar property is  12273441  and this  Score is  95.0
-----------------------------------------------------------
Based on listing reviews, for  21912251  and this  Score is  99.0
The first similar property is  21893534  and this  Score is  93.0
The second similar property is  33463937  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  22162342  and this  Score is  100.0
The first similar property is  19641296  and this  Score is  89.0
The second similar property is  6572320  and this  Score is  94.0
-----------------------

Based on listing reviews, for  19708258  and this  Score is  84.0
The first similar property is  11368302  and this  Score is  78.0
The second similar property is  14066447  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  20301087  and this  Score is  88.0
The first similar property is  20895347  and this  Score is  93.0
The second similar property is  27332423  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  14116846  and this  Score is  85.0
The first similar property is  13923395  and this  Score is  100.0
The second similar property is  16117963  and this  Score is  93.0
-----------------------------------------------------------
Based on listing reviews, for  7553565  and this  Score is  90.0
The first similar property is  35155307  and this  Score is  100.0
The second similar property is  22645576  and this  Score is  100.0
--------------------

Based on listing reviews, for  19234939  and this  Score is  92.0
The first similar property is  19155551  and this  Score is  89.0
The second similar property is  19234291  and this  Score is  90.0
-----------------------------------------------------------
Based on listing reviews, for  34571931  and this  Score is  100.0
The first similar property is  13775975  and this  Score is  100.0
The second similar property is  17539378  and this  Score is  99.0
-----------------------------------------------------------
Based on listing reviews, for  26243854  and this  Score is  100.0
The first similar property is  9508199  and this  Score is  93.0
The second similar property is  23001094  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  35545422  and this  Score is  100.0
The first similar property is  35246278  and this  Score is  93.0
The second similar property is  19967910  and this  Score is  97.0
--------------------

Based on listing reviews, for  11571554  and this  Score is  90.0
The first similar property is  30098724  and this  Score is  96.0
The second similar property is  29060134  and this  Score is  95.0
-----------------------------------------------------------
Based on listing reviews, for  35110381  and this  Score is  100.0
The first similar property is  32249929  and this  Score is  95.0
The second similar property is  21563456  and this  Score is  94.0
-----------------------------------------------------------
Based on listing reviews, for  23121624  and this  Score is  93.0
The first similar property is  35228747  and this  Score is  100.0
The second similar property is  35364919  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  18400882  and this  Score is  96.0
The first similar property is  28506238  and this  Score is  95.0
The second similar property is  647243  and this  Score is  97.0
-----------------------

Based on listing reviews, for  19508985  and this  Score is  100.0
The first similar property is  31672413  and this  Score is  85.0
The second similar property is  7754866  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  34147395  and this  Score is  100.0
The first similar property is  34819442  and this  Score is  50.0
The second similar property is  15248278  and this  Score is  100.0
-----------------------------------------------------------
Based on listing reviews, for  23398038  and this  Score is  98.0
The first similar property is  33438381  and this  Score is  100.0
The second similar property is  31484221  and this  Score is  80.0
-----------------------------------------------------------
Based on listing reviews, for  35802489  and this  Score is  100.0
The first similar property is  15129471  and this  Score is  100.0
The second similar property is  16412600  and this  Score is  99.0
------------------

In [67]:
df5_train_target = df1["score"][:lentrain]
df5_test_target = df1["score"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 50
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(df5_train, df5_train_target)
knnpreds_test = knnclf.predict(df5_test)
print (knnpreds_test)

print(classification_report(df5_test_target, knnpreds_test))
print ("Accuracy Score :",accuracy_score(df5_test_target, knnpreds_test))

[100 100 100 ... 100 100 100]
              precision    recall  f1-score   support

          20       0.00      0.00      0.00         8
          30       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         9
          50       0.00      0.00      0.00         3
          53       0.00      0.00      0.00         1
          60       0.00      0.00      0.00        20
          64       0.00      0.00      0.00         1
          67       1.00      0.33      0.50         3
          69       0.00      0.00      0.00         0
          70       0.00      0.00      0.00        15
          71       0.00      0.00      0.00         1
          73       0.00      0.00      0.00         3
          75       0.00      0.00      0.00         6
          76       0.00      0.00      0.00         3
          77       0.00      0.00      0.00         2
          78       0.00      0.00      0.00         5
          79       0.00      0.00      0.00        

C:\Users\jpran\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jpran\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
